In [1]:
import os
import shutil
import copy
import numpy as np
import pandas as pd

import argus_shapes

import skimage.io as skio

%matplotlib inline
import matplotlib.pyplot as plt

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-05-10 17:46:03,370 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
datafolder = os.path.join(os.environ['DATA_ROOT'], 'argus_shapes')

In [3]:
shutil.rmtree(datafolder)
os.mkdir(datafolder)

In [4]:
subject_params = {
    'TB': {
        'subject_id': 'S1',
        'implant_type_str': 'ArgusI',
        'implant_x': -1527,
        'implant_y': -556,
        'implant_rot': -1.13,
        'loc_od_x': 13.6,
        'loc_od_y': 0.0,
        'xmin': -36.9,
        'xmax': 36.9,
        'ymin': -36.9,
        'ymax': 36.9
    },
    '12-005': {
        'subject_id': 'S2',
        'implant_type_str': 'ArgusII',
        'implant_x': -1761,
        'implant_y': -212,
        'implant_rot': -0.188,
        'loc_od_x': 15.4,
        'loc_od_y': 1.86,
        'xmin': -30,
        'xmax': 30,
        'ymin': -22.5,
        'ymax': 22.5
    },
    '51-009': {
        'subject_id': 'S3',
        'implant_type_str': 'ArgusII',
        'implant_x': -799,
        'implant_y': 93,
        'implant_rot': -1.09,
        'loc_od_x': 15.7,
        'loc_od_y': 0.75,
        'xmin': -32.5,
        'xmax': 32.5,
        'ymin': -24.4,
        'ymax': 24.4
    },
    '52-001': {
        'subject_id': 'S4',
        'implant_type_str': 'ArgusII',
        'implant_x': -1230,
        'implant_y': 415,
        'implant_rot': -0.457,
        'loc_od_x': 15.9,
        'loc_od_y': 1.96,
        'xmin': -32,
        'xmax': 32,
        'ymin': -24,
        'ymax': 24
    }
}

In [5]:
data = []
for key, values in subject_params.items():
    values.update({'second_sight_id': key})
    data.append(values)
df = pd.DataFrame(data)
df

,implant_rot,implant_type_str,implant_x,implant_y,loc_od_x,loc_od_y,second_sight_id,subject_id,xmax,xmin,ymax,ymin
0,-0.457,ArgusII,-1230,415,15.9,1.96,52-001,S4,32.0,-32.0,24.0,-24.0
1,-1.090,ArgusII,-799,93,15.7,0.75,51-009,S3,32.5,-32.5,24.4,-24.4
2,-1.130,ArgusI,-1527,-556,13.6,0.00,TB,S1,36.9,-36.9,36.9,-36.9
3,-0.188,ArgusII,-1761,-212,15.4,1.86,12-005,S2,30.0,-30.0,22.5,-22.5


In [6]:
df.to_csv(os.path.join(datafolder, 'subjects.csv'), index=False)

In [7]:
subjects = argus_shapes.load_subjects(os.path.join(datafolder, 'subjects.csv'))
subjects.index

Index(['S4', 'S3', 'S1', 'S2'], dtype='object', name='subject_id')

In [8]:
imgfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')

features = []
targets = []
for subject in subjects.second_sight_id.unique():
    f, t = argus_shapes.load_data_raw(imgfolder, subject=subject, electrodes=None,
                                      amplitude=2.0, random_state=42,
                                      n_jobs=1, verbose=False)
    features.append(f)
    targets.append(t)
X = pd.concat(features)
y = pd.concat(targets)

In [9]:
for ssid in X.subject.unique():
    sid = subjects[subjects.second_sight_id == ssid].index[0]
    idx = X.subject == ssid
    if sum(idx) > 0:
        X.loc[idx, 'subject'] = sid

In [10]:
featuresnew = []
for (subject, date), data in X.groupby(['subject', 'date']):
    try:
        os.mkdir(os.path.join(datafolder, subject))
    except FileExistsError:
        pass
    try:
        os.mkdir(os.path.join(datafolder, subject, date))
    except FileExistsError:
        pass
    for trial, row in data.reset_index().iterrows():
        electrode = '%s%d' % (row['electrode'][0], int(row['electrode'][1:]))
        fname = '%03d_%s_%s_%.1fxTh_%.0fHz_%s.bmp' % (trial, row['subject'], electrode,
                                                      row['amp'], row['freq'], row['date'])
        fold = os.path.join(row['folder'], row['filename'])
        fnew = os.path.join(datafolder, subject, date, fname)
        shutil.copyfile(fold, fnew)
        print('Saved image as %s' % fnew)
        datarow = {
            'subject_id': subject,
            'PTS_ELECTRODE': electrode,
            'date': '%s/%s/%s' % (row['date'][:4], row['date'][4:6], row['date'][6:]),
            'PTS_FILE': os.path.join(subject, date, fname),
            'PTS_AMP': row['amp'],
            'PTS_FREQ': row['freq'],
            'PTS_PULSE_DUR': 0.45,
            'stim_class': row['stim_class']
        }
        featuresnew.append(datarow)
Xnew = pd.DataFrame(featuresnew)

Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/000_S1_C3_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/001_S1_D1_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/002_S1_D4_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/003_S1_C1_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/004_S1_C3_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/005_S1_B2_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/006_S1_C1_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/007_S1_C2_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/008_S1_C2_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1/20080702/009_S1_C2_2.0xTh_20Hz_20080702.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S1

Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/004_S3_F6_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/005_S3_B3_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/006_S3_C8_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/007_S3_F6_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/008_S3_A2_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/009_S3_B3_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/010_S3_A2_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/011_S3_F4_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/012_S3_F6_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3/20100210/013_S3_B3_2.0xTh_20Hz_20100210.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S3

Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/019_S4_F9_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/020_S4_A7_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/021_S4_D5_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/022_S4_F9_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/023_S4_F6_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/024_S4_D5_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/025_S4_D5_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/026_S4_A7_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/027_S4_F9_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4/20100204/028_S4_C8_2.0xTh_20Hz_20100204.bmp
Saved image as /home/mbeyeler/data/argus_shapes/S4

In [11]:
Xnew.to_csv(os.path.join(datafolder, 'drawings_single.csv'), index=False)

In [12]:
Xnew, ynew = argus_shapes.load_data(os.path.join(datafolder, 'drawings_single.csv'))
X.shape, Xnew.shape

((400, 10), (400, 9))

In [13]:
Xmnew, ymnew = argus_shapes.calc_mean_images(Xnew, ynew, max_area=1.5)
Xmnew.groupby('subject').count()

,amp,electrode,img_shape
subject,,,
S1,10,10,10
S2,18,18,18
S3,13,13,13
S4,19,19,19


In [14]:
[print(u) for u in Xmnew.groupby('subject').electrode.unique()]

['A4' 'B2' 'B3' 'C2' 'C3' 'C4' 'D1' 'D2' 'D3' 'D4']
['A1' 'A4' 'A6' 'A8' 'B1' 'B3' 'B9' 'C10' 'C7' 'D10' 'D7' 'E1' 'E10' 'E3'
 'E7' 'F4' 'F6' 'F9']
['A2' 'B3' 'B4' 'C1' 'C5' 'C6' 'C8' 'D3' 'E1' 'E5' 'E7' 'F4' 'F6']
['A10' 'A5' 'A7' 'A8' 'B7' 'B9' 'C10' 'C4' 'D3' 'D5' 'D7' 'E10' 'E4' 'E9'
 'F10' 'F6' 'F7' 'F8' 'F9']


[None, None, None, None]

In [15]:
Xm, ym = argus_shapes.calc_mean_images(X, y, max_area=1.5)
Xm.groupby('subject').count()

,amp,electrode,img_shape
subject,,,
S1,10,10,10
S2,18,18,18
S3,13,13,13
S4,19,19,19


In [16]:
[print(u) for u in Xm.groupby('subject').electrode.unique()]

['A4' 'B2' 'B3' 'C2' 'C3' 'C4' 'D1' 'D2' 'D3' 'D4']
['A01' 'A04' 'A06' 'A08' 'B01' 'B03' 'B09' 'C07' 'C10' 'D07' 'D10' 'E01'
 'E03' 'E07' 'E10' 'F04' 'F06' 'F09']
['A02' 'B03' 'B04' 'C01' 'C05' 'C06' 'C08' 'D03' 'E01' 'E05' 'E07' 'F04'
 'F06']
['A05' 'A07' 'A08' 'A10' 'B07' 'B09' 'C04' 'C10' 'D03' 'D05' 'D07' 'E04'
 'E09' 'E10' 'F06' 'F07' 'F08' 'F09' 'F10']


[None, None, None, None]

In [17]:
argus_shapes.load_subjects(os.path.join(datafolder, 'subjects.csv'))

,implant_rot,implant_x,implant_y,loc_od_x,loc_od_y,second_sight_id,xrange,yrange,implant_type
subject_id,,,,,,,,,
S4,-0.457,-1230,415,15.9,1.96,52-001,"(-32.0, 32.0)","(-24.0, 24.0)",<class 'pulse2percept.implants.ArgusII'>
S3,-1.090,-799,93,15.7,0.75,51-009,"(-32.5, 32.5)","(-24.4, 24.4)",<class 'pulse2percept.implants.ArgusII'>
S1,-1.130,-1527,-556,13.6,0.00,TB,"(-36.9, 36.9)","(-36.9, 36.9)",<class 'pulse2percept.implants.ArgusI'>
S2,-0.188,-1761,-212,15.4,1.86,12-005,"(-30.0, 30.0)","(-22.5, 22.5)",<class 'pulse2percept.implants.ArgusII'>
